In [1]:
import pandas as pd
import re

In [12]:
og_file = '../ppi_ml/data/og_proteomes/nog_mapping/human.euNOG.diamond.mapping.2759'
dis_file = '../human_disease/human_disease_groups_manuallyedited.csv'
res_file = '../ppi_ml/results/final/interactome_051023.csv'
cutoff = 'cut_796'
outfile = '../ppi_ml/results/final/interactome_diseases_051023'

In [3]:
def fmt_ids(x):
    prot_id = re.search("(?<=\|).*(?=\|)", x)[0]
    return(prot_id)

In [4]:
dis = pd.read_csv(dis_file)
dis = dis[['entry', 'gene_names_primary', 'label']]

,entry,gene_names_primary,label
0,P35555,FBN1,Marfanoid-progeroid-lipodystrophy syndrome
1,P07954,FH,Hereditary leiomyomatosis
2,Q16698,DECR1,"2,4-dienoyl-CoA reductase deficiency"
3,Q4G0N4,NADK2,"2,4-dienoyl-CoA reductase deficiency"
4,Q9UDR5,AASS,"2,4-dienoyl-CoA reductase deficiency"
...,...,...,...
5755,O95259,KCNH1,Zimmermann-Laband syndrome
5756,P21281,ATP6V1B2,Zimmermann-Laband syndrome
5757,Q9UGI6,KCNN3,Zimmermann-Laband syndrome
5758,Q9BRI3,SLC30A2,Zinc deficiency


In [5]:
ogs = pd.read_csv(og_file, sep='\t')
ogs['entry'] = [fmt_ids(i) for i in ogs['ProteinID']]
ogs = ogs[['ID','entry']]

In [6]:
mapped = ogs.merge(dis, how='inner')
dis_grp = mapped.groupby('ID', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))

In [7]:
res = pd.read_csv(res_file)
res_mapped = res.merge(dis_grp, how='left')

In [8]:
dis_rows = res_mapped[~res_mapped.label.isnull()]
clusters = dis_rows[cutoff].unique()

In [14]:
out_df = res_mapped[res_mapped[cutoff].isin(clusters)]
out_df.to_csv(outfile+'.csv', index=False)
out_df.to_excel(outfile+'.xlsx', index=False)